In [1]:
import wandb
import configparser
import torch
import pprint
import torch.nn as nn
from torchvision import datasets, transforms
import torch.nn.functional as F
import wandb

In [2]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(DEVICE)

cuda


## Sweep config 만들기

In [3]:
#어떤 방법으로 최적의 파라미터를 찾을 것인가?
sweep_config = {
    'method': 'bayes'
    }

#어떤 metric을, 어떤 기준으로(minimize or maximize) 좋다고 판단할 것인가?
metric = {
    'name': 'Test_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [4]:
#기타 비교할 optimizer 및 모델의 파라미터들을 입력
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.2, 0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

In [8]:
#바꿔가며 실험할 건 아닌데 config에 넣고 싶을때는 'value'를 직접적으로 입력
parameters_dict.update({
    'epochs': {
        'value': 20}
    })


In [9]:
#실험할 파라미터들을 추가추가
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 1e-6,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

sweep_config['parameters'] = parameters_dict

In [10]:
#파라미터 가시화
pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'Test_loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'dropout': {'values': [0.2, 0.3, 0.4, 0.5]},
                'epochs': {'value': 20},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 1e-06},
                'optimizer': {'values': ['adam', 'sgd']}}}


## Run sweep agent

In [11]:
sweep_id = wandb.sweep(sweep_config, project="CIFAR-10-classification_sweep")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: dorn02an
Sweep URL: https://wandb.ai/jd-hwang/CIFAR-10-classification_sweep/sweeps/dorn02an


In [12]:
class CNN(nn.Module):
    def __init__(self,fc_layer_size,dropout):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=8,
            kernel_size=3,
            padding=1)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            padding=1)
        self.pool = nn.MaxPool2d(
            kernel_size=2,
            stride=2
        )
        self.fc1 = nn.Linear(8 * 8 * 16, fc_layer_size)
        self.fc2 = nn.Linear(fc_layer_size, 32)
        self.fc3 = nn.Linear(32, 10)
        self.do1 = nn.Dropout(p=dropout)
        self.do2 = nn.Dropout(p=dropout)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool(x)

        x = x.view(-1, 8 * 8 * 16)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.do1(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.do2(x)
        x = self.fc3(x)
        x = torch.log_softmax(x, dim=1)
        return x

In [13]:
def build_dataset(batch_size):
   
    # download MNIST training dataset
    train_dataset = datasets.CIFAR10(root="./data/",
                                    train=True,
                                    download=True,
                                    transform=transforms.ToTensor())

    test_dataset = datasets.CIFAR10(root="./data/",
                                    train=False,
                                    download=True,
                                    transform=transforms.ToTensor())

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, test_loader


def build_network(fc_layer_size, dropout):
    network = CNN(fc_layer_size,dropout)
    return network.to(DEVICE)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = torch.optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(model, train_loader, optimizer, criterion):
    model.train()
    train_loss=0
    correct =0
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
    
        train_loss+=loss.item()
        prediction = output.max(1, keepdim=True)[1]
        correct += prediction.eq(label.view_as(prediction)).sum().item()
    train_loss /= len(train_loader.dataset)
    train_accuracy = 100. * correct / len(train_loader.dataset)
    return train_loss, train_accuracy

def evaluate(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [14]:
def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        train_loader,test_lodaer = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(config.epochs):
            train_loss,train_accuracy = train_epoch(network, train_loader, optimizer,criterion)
            test_loss,test_accuracy = evaluate(network, test_lodaer, criterion)
            wandb.log({"Train_loss":train_loss,"Train_acc":train_accuracy,"Test_loss":test_loss,"Test_acc":test_accuracy})  

In [15]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: ohq2hzo2 with config:
wandb: 	batch_size: 184
wandb: 	dropout: 0.4
wandb: 	epochs: 20
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.03780970552991392
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jd-hwang. Use `wandb login --relogin` to force relogin


Files already downloaded and verified
Files already downloaded and verified


Test_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_loss,▁▅▆▇▇▇▇█████████████
Train_acc,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_acc,10.0
Test_loss,0.01268
Train_acc,9.786
Train_loss,0.01254


wandb: Agent Starting Run: tbxtj17f with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.4
wandb: 	epochs: 20
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.051954322884376784
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Files already downloaded and verified
Files already downloaded and verified


...

Files already downloaded and verified
Files already downloaded and verified


Test_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_loss,▁▄▅▆▇▇▇▇████████████
Train_acc,█▁▂▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_acc,10.0
Test_loss,0.00968
Train_acc,9.804
Train_loss,0.00963


wandb: Agent Starting Run: lklyw17x with config:
wandb: 	batch_size: 248
wandb: 	dropout: 0.3
wandb: 	epochs: 20
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.02380651320410078
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Files already downloaded and verified
Files already downloaded and verified


Test_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_loss,▁▄▄▃▆▆▆▆▇▇▇▇▇███████
Train_acc,▁▅▅▅▂▃▅▆▇▇██████████
Train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_acc,10.0
Test_loss,0.00944
Train_acc,9.972
Train_loss,0.00931
